In [36]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV, train_test_split
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans
from sklearn.discriminant_analysis import StandardScaler
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.svm import SVC
from sklearn.feature_selection import VarianceThreshold
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import f1_score



In [37]:
df_hippo_original = pd.read_csv('/home/cid34senhas/Desktop/DAATP/Entrega/1_DF_CN-MCI_1_After_Pre_Processamento.csv') 
df_ocipital_original = pd.read_csv('/home/cid34senhas/Desktop/DAATP/Entrega/1_Occipital_pre_processamento.csv') 

print("Shape Hippo:")
print(df_hippo_original.shape)

print("Shape Occipital:")
print(df_ocipital_original.shape)

Shape Hippo:
(305, 2014)
Shape Occipital:
(305, 2026)


In [38]:
def remove_highly_correlated_features(data, target, threshold):
    """
    Remove features altamente correlacionadas, mantendo a que tem maior correlação com o target.

    Args:
        data (pd.DataFrame): DataFrame contendo as features.
        target (pd.Series): Coluna com os valores do target.
        threshold (float): Valor de correlação acima do qual consideramos features altamente correlacionadas.

    Returns:
        pd.DataFrame: DataFrame com as colunas altamente correlacionadas removidas.
        set: Conjunto das colunas removidas.
    """
    # Compute the correlation matrix
    corr_matrix = data.corr().abs()
    
    # Compute correlation of each feature with the target
    target_corr = data.corrwith(data['Transition']).abs()
    
    # Identify columns to remove based on correlation
    to_drop = set()
    for i in range(len(corr_matrix.columns)):
        for j in range(i + 1, len(corr_matrix.columns)):
            if corr_matrix.iloc[i, j] >= threshold:
                col_i = corr_matrix.columns[i]
                col_j = corr_matrix.columns[j]
                
                # Remove the column with lower correlation with the target
                if target_corr[col_i] < target_corr[col_j]:
                    to_drop.add(col_i)
                else:
                    to_drop.add(col_j)

    # Drop the identified columns
    data_final = data.drop(columns=to_drop)

    return data_final, to_drop



def remove_low_variance_features(data, threshold):
    """
    Remove columns with a standard deviation below the given threshold.

    Parameters:
        data (pd.DataFrame): The input DataFrame.
        threshold (float): The minimum standard deviation a column must have to be retained.

    Returns:
        tuple: A tuple containing:
            - pd.DataFrame: A DataFrame with low-variance columns removed.
            - list: A list of columns that were removed.
    """
    # Calculate the standard deviation for each column
    std_dev = data.std()

    # Identify columns to keep (standard deviation above threshold)
    retained_columns = std_dev[std_dev > threshold].index

    # Identify columns to remove
    removed_columns = std_dev[std_dev <= threshold].index

    # Create a new DataFrame with only the retained columns
    data_reduced = data[retained_columns]

    return data_reduced, removed_columns

## Random Forest

In [39]:
df_hippo_Randon_Forest = pd.read_csv('/home/cid34senhas/Desktop/DAATP/Entrega/2_data_Random_Forest_Clean.csv')

X_cleaned, to_drop = remove_highly_correlated_features(df_ocipital_original, 'Transition', 0.81)

data_occipital_rf = df_ocipital_original.drop(columns=to_drop,axis=1)

data_, to_drop_2 = remove_low_variance_features(X_cleaned, 0.005)

data_ocipital_final = data_occipital_rf.drop(columns=to_drop_2,axis=1, errors='ignore')

print("Shape Do data_set Final do Hippocampus  ")
print(df_hippo_Randon_Forest.shape)
print("\n")
print("Shape Do data_set Final do Occipital ")
print(data_ocipital_final.shape)


Shape Do data_set Final do Hippocampus  
(305, 348)


Shape Do data_set Final do Occipital 
(305, 354)


### Resultados Occipital

In [40]:

X_ocipital = data_ocipital_final.drop('Transition', axis=1)
y_ocipital = data_ocipital_final['Transition']


rf_ocipital = RandomForestClassifier(n_estimators=143, max_features = 'sqrt', max_depth= 24, min_samples_split=32, min_samples_leaf=13, bootstrap=False,class_weight = 'balanced',random_state=34)
rf_ocipital_score = cross_val_score(rf_ocipital, X_ocipital, y_ocipital, cv=5,scoring='f1_macro')
print(f"Cross Validation Score: {rf_ocipital_score}")
print(f"Cross Validation Score Média: {rf_ocipital_score.mean()}")
print(f"Cross Validation Score STD: {rf_ocipital_score.std()}")

Cross Validation Score: [0.17580618 0.17794058 0.25709866 0.24403342 0.26589744]
Cross Validation Score Média: 0.22415525626289048
Cross Validation Score STD: 0.03923327292913626


## XGBOOST 

In [41]:
df_hippo_xgboost = pd.read_csv('/home/cid34senhas/Desktop/DAATP/Entrega/2_data_XGBoost.csv')

X_cleaned, to_drop = remove_highly_correlated_features(df_ocipital_original, 'Transition', 0.82)

data_occipital_xg = df_ocipital_original.drop(columns=to_drop,axis=1)


print("Shape Do data_set Final do Hippocampus  ")
print(df_hippo_xgboost.shape)
print("\n")
print("Shape Do data_set Final do Occipital ")
print(data_occipital_xg.shape)


Shape Do data_set Final do Hippocampus  
(305, 412)


Shape Do data_set Final do Occipital 
(305, 414)


In [42]:

X_ocipital_xg = data_occipital_xg.drop('Transition', axis=1)
y_ocipital_xg = data_occipital_xg['Transition']


### Resultados Occipital

In [43]:
xgboost_model = XGBClassifier(n_estimators= 846, max_depth=  5, learning_rate= 0.1660773939789664, subsample= 0.9293091645826319, colsample_bytree =  0.5898888173202074, gamma =0.0035106908632086685, reg_alpha =  3.3065806009993237e-06, reg_lambda = 0.6447155795041164, min_child_weight= 7)


xgboost_score_ocii = cross_val_score(xgboost_model, X_ocipital_xg, y_ocipital_xg, cv=5,scoring='f1_macro')

print(f"Cross Validation Score: {xgboost_score_ocii}")
print(f"Cross Validation Score Média: {xgboost_score_ocii.mean()}")
print(f"Cross Validation Score STD: {xgboost_score_ocii.std()}")

Cross Validation Score: [0.20342171 0.21553805 0.23539686 0.29826134 0.22008276]
Cross Validation Score Média: 0.2345401461823901
Cross Validation Score STD: 0.033465138375488875


## Extra TRees 

In [44]:
df_hippo_extra_trees = pd.read_csv('/home/cid34senhas/Desktop/DAATP/Entrega/2_data_ExtraTrees.csv')

X_cleaned, to_drop = remove_highly_correlated_features(df_ocipital_original, 'Transition', 0.93)

data_occipital_xg = df_ocipital_original.drop(columns=to_drop,axis=1)


print("Shape Do data_set Final do Hippocampus  ")
print(df_hippo_extra_trees.shape)
print("\n")
print("Shape Do data_set Final do Occipital ")
print(data_occipital_xg.shape)


Shape Do data_set Final do Hippocampus  
(305, 812)


Shape Do data_set Final do Occipital 
(305, 743)


In [45]:

X_ocipital_xg = data_occipital_xg.drop('Transition', axis=1)
y_ocipital_xg = data_occipital_xg['Transition']


In [46]:
extra_trees_model = ExtraTreesClassifier( n_estimators=111, max_features='sqrt',max_depth=54, min_samples_split=14, min_samples_leaf=4 , bootstrap=False, class_weight='balanced',criterion='gini' , random_state=34)


extra_trees_score_ocii = cross_val_score(extra_trees_model, X_ocipital_xg, y_ocipital_xg, cv=5,scoring='f1_macro')

print(f"Cross Validation Score: {extra_trees_score_ocii}")
print(f"Cross Validation Score Média: {extra_trees_score_ocii.mean()}")
print(f"Cross Validation Score STD: {extra_trees_score_ocii.std()}")

Cross Validation Score: [0.25083061 0.15461747 0.25568173 0.22645687 0.21468013]
Cross Validation Score Média: 0.22045336274387486
Cross Validation Score STD: 0.036242384876576


## Gradient Boosting

In [50]:
df_hippo_gradient_boost = pd.read_csv('/home/cid34senhas/Desktop/DAATP/Entrega/2_data_GradientBoost.csv')

X_cleaned, to_drop = remove_highly_correlated_features(df_ocipital_original, 'Transition', 0.83)

data_occipital_xg = df_ocipital_original.drop(columns=to_drop,axis=1)


print("Shape Do data_set Final do Hippocampus  ")
print(df_hippo_gradient_boost.shape)
print("\n")
print("Shape Do data_set Final do Occipital ")
print(data_occipital_xg.shape)


Shape Do data_set Final do Hippocampus  
(305, 434)


Shape Do data_set Final do Occipital 
(305, 432)


In [51]:

X_ocipital_xg = data_occipital_xg.drop('Transition', axis=1)
y_ocipital_xg = data_occipital_xg['Transition']


In [52]:
gb_model_norm = GradientBoostingClassifier(learning_rate=0.04471222414633008, n_estimators=109, max_depth=3, min_samples_split=18, min_samples_leaf=1, subsample=1.0, max_features='sqrt',random_state=34)


gb_socre = cross_val_score(gb_model_norm, X_ocipital_xg, y_ocipital_xg, cv=5,scoring='f1_macro')

print(f"Cross Validation Score: {gb_socre}")
print(f"Cross Validation Score Média: {gb_socre.mean()}")
print(f"Cross Validation Score STD: {gb_socre.std()}")

Cross Validation Score: [0.20585895 0.25323459 0.18225108 0.28791248 0.25851371]
Cross Validation Score Média: 0.23755416213936426
Cross Validation Score STD: 0.03816845732774831


| Modelo           | **Média (Hippo)** | **Desvio Padrão (Hippo)** | **Média (Occipital)** | **Desvio Padrão (Occipital)** | **Diferença (Hippo - Occipital)** |
|:----------------:|:-----------------:|:-------------------------:|:---------------------:|:-----------------------------:|:---------------------------------:|
| Random Forest    |      **0.3952**   |          **0.0625**       |       **0.2241**      |           **0.0392**          |             **0.1711**            |
| XGBoost          |      **0.3630**   |          **0.0495**       |       **0.2345**      |           **0.03346**         |             **0.1285**            |
| ExtraTrees       |      **0.3676**   |          **0.0533**       |       **0.2204**      |           **0.036**           |             **0.1472**            |
| GradientBoost    |      **0.3736**   |          **0.0696**       |       **0.2375**      |           **0.038**           |             **0.1361**            |
